# XtEHR Logical model processing

* ophalen XtEHR Logical Models
* consistente layout met zibs
* klaar voor een mapping - zie mapping Notebook

## Relevante kern content

NL kernset:
- Behandelaanwijzing

ePS required:
- Medicatie
- Diagnose/ Probleemlijst(actief)
- Verrichtingen(<6mnd, ≥ 6mnd)
- Allergieën en intoleranties + - - 
- Alerts
- Hulpmiddel
- Patiënt

ePS recommended + optional:
- Vaccinations
- Implants
- Treatment restrictions
- Vital signs
- History of illness
- Pregnancy
- Social history
- Functional status
- Results
- Plan of Care
- Encounters

# Imports

In [1]:
import pandas as pd
import requests
import json
import os
from io import StringIO

# Download JSON

* Dit hoeft maar een keer te gebeuren, bij nieuwe mappings download de JSON die nodig is. 
* Kan ook door directe download 'raw' JSON.
* Dit zijn de XtEHR Logical Models

In [3]:
xtehrlist = ['Address', 'AlertFlag', 'AllergyIntolerance', 'BodyStructure', 'Condition', 'Device','DeviceUse', 'HealthProfessional', 'HumanName', 'Location', 'Medication', 'Organization', 'Patient', 'Procedure', 'Telecom']
xtehrbase = 'https://build.fhir.org/ig/Xt-EHR/xt-ehr-common/StructureDefinition-EHDS'
# res = 'AlertFlag'
if (not os.path.exists('xtehr')):
    os.makedirs('xtehr')
for res in xtehrlist:
    print(res)
    url = xtehrbase + res + '.json'
    response = requests.get(url)
    jsondata = response.json()
    with open(os.path.join('xtehr', 'downloads', res + '.json'), 'w') as jsonfile:
        json.dump(jsondata, jsonfile, indent=4)

Address
AlertFlag


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

# Kies en lees het te verwerken Logical Model

* Toon en verifieer dat de uitvoer er is

In [12]:
base = 'xtehr'
file = 'Organization'
url = os.path.join(base, 'downloads', file + '.json')
with open(url) as jsonfile:
    xlm = json.load(jsonfile)['snapshot']['element']
dfx = pd.DataFrame(xlm)
dfx

,id,path,short,definition,min,max,base,isModifier,mapping,type,binding
0,EHDSOrganization,EHDSOrganization,Organization model,C.3 - EHDS refined base model for Health provi...,0,*,"{'path': 'Base', 'min': 0, 'max': '*'}",False,"[{'identity': 'rim', 'map': 'n/a'}]",NaN,NaN
1,EHDSOrganization.identifier,EHDSOrganization.identifier,C.3.1 - Identifier,Health provider organisation identifier,0,*,"{'path': 'EHDSOrganization.identifier', 'min':...",NaN,NaN,[{'code': 'Identifier'}],NaN
2,EHDSOrganization.type,EHDSOrganization.type,C.3.2 - Type,Kind of organization,0,*,"{'path': 'EHDSOrganization.type', 'min': 0, 'm...",NaN,NaN,[{'code': 'CodeableConcept'}],"{'strength': 'preferred', 'description': 'HL7 ..."
3,EHDSOrganization.name,EHDSOrganization.name,C.3.3 - Name,Health provider organisation name,0,1,"{'path': 'EHDSOrganization.name', 'min': 0, 'm...",NaN,NaN,[{'code': 'string'}],NaN
4,EHDSOrganization.address,EHDSOrganization.address,C.3.4 - Address,Mailing and home or office addresses. The addr...,0,*,"{'path': 'EHDSOrganization.address', 'min': 0,...",NaN,NaN,[{'code': 'https://www.xt-ehr.eu/specification...,NaN
5,EHDSOrganization.telecom,EHDSOrganization.telecom,C.3.5 - Telecom,Telecommunication contact information (address...,0,*,"{'path': 'EHDSOrganization.telecom', 'min': 0,...",NaN,NaN,[{'code': 'https://www.xt-ehr.eu/specification...,NaN
6,EHDSOrganization.partOf,EHDSOrganization.partOf,C.3.6 - Part of,The organization of which this organization fo...,0,1,"{'path': 'EHDSOrganization.partOf', 'min': 0, ...",NaN,NaN,"[{'code': 'Reference', 'targetProfile': ['http...",NaN


# Opschonen uitvoer zodat zib en xtehr consistent zijn

* een card. veld met n..m
* type (is een JSON object) omzetten naar tekst
* alleen nodige kolommen houden

In [13]:
dfx.insert(2, 'card.', None)
dfx['card.'] = dfx['min'].astype(str) + '..' + dfx['max'].astype(str)
for i, row in dfx.iterrows():
    if 'type' in row:
        # row['type'][0]['code']
        try:
            dfx.at[i, 'type'] = row['type'][0]['code'].split('/')[-1]
        except TypeError:
            pass
dfx= dfx[['id', 'path', 'short', 'definition', 'type', 'card.', 'binding']]
dfx

,id,path,short,definition,type,card.,binding
0,EHDSOrganization,EHDSOrganization,Organization model,C.3 - EHDS refined base model for Health provi...,NaN,0..*,NaN
1,EHDSOrganization.identifier,EHDSOrganization.identifier,C.3.1 - Identifier,Health provider organisation identifier,Identifier,0..*,NaN
2,EHDSOrganization.type,EHDSOrganization.type,C.3.2 - Type,Kind of organization,CodeableConcept,0..*,"{'strength': 'preferred', 'description': 'HL7 ..."
3,EHDSOrganization.name,EHDSOrganization.name,C.3.3 - Name,Health provider organisation name,string,0..1,NaN
4,EHDSOrganization.address,EHDSOrganization.address,C.3.4 - Address,Mailing and home or office addresses. The addr...,EHDSAddress,0..*,NaN
5,EHDSOrganization.telecom,EHDSOrganization.telecom,C.3.5 - Telecom,Telecommunication contact information (address...,EHDSTelecom,0..*,NaN
6,EHDSOrganization.partOf,EHDSOrganization.partOf,C.3.6 - Part of,The organization of which this organization fo...,Reference,0..1,NaN


# Opslaan in Excel

In [14]:
url = os.path.join(base, 'mappable', file + '.xlsx')
dfx.to_excel(url, index=False)